In [1]:
import pandas as pd
import requests
import time
import dateutil
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
     

In [2]:
API_KEY = 'xxxxxxxxxxxxxxxxxxxxxxxx' #Your API key here
# Change start and end date if you wish
start = (1986, 1)
end = (2023, 12)
start = list(start)
end = list(end)

In [3]:
def send_request(date):
    base_url = 'https://api.nytimes.com/svc/archive/v1/'
    url = base_url + '/' + str(date[0]) + '/' + str(date[1]) + '.json?api-key=' + API_KEY
    query = requests.get(url)
    response = query.json()
    time.sleep(12)
    return response

def parse_response(response):
    data = {'headline': [],  
        'date': [], 
        'web_url': [],
        'doc_type': [],
        'lead_paragraph': [],
        'material_type': [],
        'author': [],
        'section': [],
        'subsection': [],
        'keywords': []}
    
    articles = response['response']['docs'] 
    for article in articles: 
        date = dateutil.parser.parse(article['pub_date']).date()
        data['date'].append(date)
        data['headline'].append(article['headline']['main']) 
        if 'section_name' in article:
            data['section'].append(article['section_name'])
        else:
            data['section'].append(None)
        if 'lead_paragraph' in article:
            data['lead_paragraph'].append(article['lead_paragraph'])
        else:
            data['lead_paragraph'].append(None)
        if 'web_url' in article:
            data['web_url'].append(article['web_url'])
        else:
            data['web_url'].append(None)
        if 'subsection_name' in article:
            data['subsection'].append(article['subsection_name'])
        else:
            data['subsection'].append(None)
        if 'byline' in article:
            data['author'].append(article['byline']['original'])
        else:
            data['author'].append(None)
        data['doc_type'].append(article['document_type'])
        if 'type_of_material' in article: 
            data['material_type'].append(article['type_of_material'])
        else:
            data['material_type'].append(None)
        keywords = [keyword['value'] for keyword in article['keywords'] if keyword['name'] == 'subject']
        data['keywords'].append(keywords)
    return pd.DataFrame(data) 



def get_data(start,end):
    df1 = []
    df2 = []
    df2 =  pd.DataFrame.from_dict(df2)
    date = start
    while (date[0] <= end[0] and date[1] <= end[1]):
            df1 = send_request(date)
            df1 = parse_response(df1)
            df2 = pd.concat([df2 ,df1])
            if date[1] == 12:
                 print(date[0])
                 date[0]+=1
                 date[1]= 1
            else:
                 date[1]+=1
            
    return df2

In [4]:
downloaded = get_data(start,end)


1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023


In [5]:
downloaded.to_csv('filewhole.csv')

In [8]:
filtered = downloaded[downloaded['headline'].str.contains(' AI |Artificial Intelligence|robot| ai |Robot|artificial intelligence|AI ', na=False)]
filtered.shape 


(1625, 10)

In [10]:
downloaded.head(2)

,headline,date,web_url,doc_type,lead_paragraph,material_type,author,section,subsection,keywords
0,Sex Guide Is Recalled For Correction of Text,1986-01-01,https://www.nytimes.com/1986/01/01/books/sex-g...,article,"Because of what it termed ''a serious error,''...",News,,Books,None,"[Sex, Children and Youth, Books and Literature..."
1,A WOODEN VESSEL FOR PROOFING BREAD,1986-01-01,https://www.nytimes.com/1986/01/01/garden/kitc...,article,NOTHING is as soul-warming this time of year a...,News,By Pierre Franey,Home & Garden,None,[]


In [ ]:

#export to csv
combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')